# Segmenting and Clustering Neighborhoods in Toronto (Q3)

In [1]:
!pip install beautifulsoup4
##parse html
!pip install lxml 
!pip install html5lib
#request library
!pip install requests
!pip install geopy
!pip install folium 

### 1. Import the necessary package

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # for map randering
import numpy as np # library to handle data in a vectorized manner
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### 2. Set parameter and inititate web scraping using pandas looking for table

In [49]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [50]:
tables = pd.read_html(url)
print('Tables found:', len(tables))

Tables found: 3


In [51]:
df = tables[0]  #we identified the first table is the one that we are looking for
df.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

### 4. Cleanup the extracted table

In [52]:
#rename column 'postal code' for easy referenec
df = df.rename(columns={'Postal Code': 'PostalCode'})
#drop 'not assigned' borough
df = df[df.Borough != 'Not assigned']
#reset index
df.reset_index(drop=True, inplace=True)
#View the first 12 lines
df.head(12)

PostalCode           Borough  \
0         M3A        North York   
1         M4A        North York   
2         M5A  Downtown Toronto   
3         M6A        North York   
4         M7A  Downtown Toronto   
5         M9A         Etobicoke   
6         M1B       Scarborough   
7         M3B        North York   
8         M4B         East York   
9         M5B  Downtown Toronto   
10        M6B        North York   
11        M9B         Etobicoke   

                                        Neighbourhood  
0                                           Parkwoods  
1                                    Victoria Village  
2                           Regent Park, Harbourfront  
3                    Lawrence Manor, Lawrence Heights  
4         Queen's Park, Ontario Provincial Government  
5             Islington Avenue, Humber Valley Village  
6                                      Malvern, Rouge  
7                                           Don Mills  
8                     Parkview Hill, Woodbine Gardens  
9                            Garden District, Ryerson  
10                                          Glencairn  
11  West Deane Park, Princess Gardens, Martin Grov...

In [53]:
# Dimenstion of the table as requested in the question
df.shape

(103, 3)

### 5. Import CSV file containing longitude and latitude

In [54]:
longlat_url='https://cocl.us/Geospatial_data'

In [55]:
df2= pd.read_csv(longlat_url)
df2.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

### 6. Merge the longitude latitude into existing table

In [56]:
df = pd.merge(df, df2, 
                     left_on = 'PostalCode', 
                     right_on = 'Postal Code', 
                     how='left')

df.head(12)

PostalCode           Borough  \
0         M3A        North York   
1         M4A        North York   
2         M5A  Downtown Toronto   
3         M6A        North York   
4         M7A  Downtown Toronto   
5         M9A         Etobicoke   
6         M1B       Scarborough   
7         M3B        North York   
8         M4B         East York   
9         M5B  Downtown Toronto   
10        M6B        North York   
11        M9B         Etobicoke   

                                        Neighbourhood Postal Code   Latitude  \
0                                           Parkwoods         M3A  43.753259   
1                                    Victoria Village         M4A  43.725882   
2                           Regent Park, Harbourfront         M5A  43.654260   
3                    Lawrence Manor, Lawrence Heights         M6A  43.718518   
4         Queen's Park, Ontario Provincial Government         M7A  43.662301   
5             Islington Avenue, Humber Valley Village         M9A  43.667856   
6                                      Malvern, Rouge         M1B  43.806686   
7                                           Don Mills         M3B  43.745906   
8                     Parkview Hill, Woodbine Gardens         M4B  43.706397   
9                            Garden District, Ryerson         M5B  43.657162   
10                                          Glencairn         M6B  43.709577   
11  West Deane Park, Princess Gardens, Martin Grov...         M9B  43.650943   

    Longitude  
0  -79.329656  
1  -79.315572  
2  -79.360636  
3  -79.464763  
4  -79.389494  
5  -79.532242  
6  -79.194353  
7  -79.352188  
8  -79.309937  
9  -79.378937  
10 -79.445073  
11 -79.554724

In [57]:
df.shape #-To check on dimension of the table

(103, 6)

### 7. Drop unnecessary column

In [58]:
df = df.drop(['Postal Code'], axis = 1) #Drop Postal Code column which is redundant
df.head()

PostalCode           Borough                                Neighbourhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [59]:
df.shape #check new table dimension

(103, 5)

### 8. Filter Borough with 'Toronto' value inside

In [60]:
filter = 'Toronto'

In [61]:
df=df.loc[df['Borough'].str.contains('Toronto')]
df.head()

PostalCode           Borough                                Neighbourhood  \
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9         M5B  Downtown Toronto                     Garden District, Ryerson   
15        M5C  Downtown Toronto                               St. James Town   
19        M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  
2   43.654260 -79.360636  
4   43.662301 -79.389494  
9   43.657162 -79.378937  
15  43.651494 -79.375418  
19  43.676357 -79.293031

In [62]:
df.reset_index(drop=True, inplace=True) #reset index
df.head()

PostalCode           Borough                                Neighbourhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

In [63]:
df.shape #check the new shape

(39, 5)

### 9. Use geopy to get longitude and latitude of Toronto, Canada

In [64]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 10. Create Toronto Map using folium 

In [65]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
# The zip() function returns a zip object, 
# which is an iterator of tuples where the first item in each passed iterator is paired together, 
# and then the second item in each passed iterator are paired together etc.
# If the passed iterators have different lengths, the iterator with the least items decides the length of the new iterator.
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 11. Define Foursquare Credentials

In [66]:
CLIENT_ID = 'X1SOLSFVG0MVLANAZ1J2GFGEXSZGJBRWUFZSWBSPN3A5GR1P' 
CLIENT_SECRET = 'U1SNGOWOI2SHTAZLZOXYEUESIRL0KIL3SKP3D2AEALS0FYLN'
VERSION = '20180605'
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X1SOLSFVG0MVLANAZ1J2GFGEXSZGJBRWUFZSWBSPN3A5GR1P
CLIENT_SECRET:U1SNGOWOI2SHTAZLZOXYEUESIRL0KIL3SKP3D2AEALS0FYLN


### 12. Exploring various venues in Toronto

In [88]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [89]:
#Checking the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1611, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0           Tandem Coffee       43.653559       -79.361809   
1        Roselle Desserts       43.653447       -79.362017   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4          Impact Kitchen       43.656369       -79.356980   

        Venue Category  
0          Coffee Shop  
1               Bakery  
2  Distribution Center  
3                  Spa  
4           Restaurant

-- The result come out with 1611 lines. 

Dropping Venue Latitude and Venue Longitude column as we are not using them.

In [90]:
toronto_venues = toronto_venues.drop(['Venue Latitude','Venue Longitude'], axis = 1)

Checking the size of the dataframe

In [91]:
print(toronto_venues.shape)
toronto_venues.head()

(1611, 5)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue       Venue Category  
0           Tandem Coffee          Coffee Shop  
1        Roselle Desserts               Bakery  
2  Cooper Koo Family YMCA  Distribution Center  
3     Body Blitz Spa East                  Spa  
4          Impact Kitchen           Restaurant

-- There is 1,611 lines representing unique vennue in each neigborrhood

Checking the number of unique categories 

In [92]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 230 uniques categories.


Checking number iof venues grouped by each neigbourhood

In [93]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            56   
Brockton, Parkdale Village, Exhibition Place                           22   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     14   
Central Bay Street                                                     63   
Christie                                                               16   
Church and Wellesley                                                   78   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             38   
Davisville North                                                       10   
Dufferin, Dovercourt Village                                           19   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
India Bazaar, The Beaches West                                         20   
Kensington Market, Chinatown, Grange Park                              65   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               42   
Moore Park, Summerhill East                                             4   
North Toronto West, Lawrence Park                                      19   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            31   
Regent Park, Harbourfront                                              47   
Richmond, Adelaide, King                                               92   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     36   
St. James Town                                                         81   
St. James Town, Cabbagetown                                            46   
Stn A PO Boxes                                                         98   
Studio District                                                        36   
Summerhill West, Rathnelly, South Hill, Forest ...                     14   
The Annex, North Midtown, Yorkville                                    19   
The Beaches                                                             4   
The Danforth West, Riverdale                                           42   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         30   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             56   
Brockton, Parkdale Village, Exhibition Place                            22   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      14   
Central Bay Street                                                      63   
Christie                                                                16   
Church and Wellesley                                                    78   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              38   
Davisville North                    

### 13. Analyzing each neighboorhood

#### one hot encoding

In [94]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# get _dummies= Convert categorical variable into dummy/indicator variables.

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Adult Boutique  Airport  Airport Food Court  Airport Gate  \
0            0               0        0                   0             0   
1            0               0        0                   0             0   
2            0               0        0                   0             0   
3            0               0        0                   0             0   
4            0               0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Theater  Theme Restaurant  Tibetan Restaurant  \
0             0  ...        0                 0                   0   
1             0  ...        0                 0                   0   
2             0  ...        0                 0                   0   
3             0  ...        0                 0                   0   
4             0  ...        0                 0                   0   

   Toy / Game Store  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0                 0      0              0                              0   
1                 0      0              0                              0   
2                 0      0              0                              0   
3                 0      0              0                              0   
4                 0      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Wine Bar  
0                 0                      0         0  
1                 0                      0         0  
2                 0                      0         0  
3                 0                      0         0  
4                 0                      0         0  

[5 rows x 230 columns]

In [95]:
toronto_onehot.shape #examine the size of the dataframe

(1611, 230)

#### Group the neighborhood and find mean for each category

In [96]:
toronto_group = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_group

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1        Brockton, Parkdale Village, Exhibition Place     0.000000   
2   Business reply mail Processing Centre, South C...     0.062500   
3   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                  Central Bay Street     0.015873   
5                                            Christie     0.000000   
6                                Church and Wellesley     0.025641   
7                      Commerce Court, Victoria Hotel     0.000000   
8                                          Davisville     0.000000   
9                                    Davisville North     0.000000   
10                       Dufferin, Dovercourt Village     0.000000   
11             First Canadian Place, Underground city     0.000000   
12    Forest Hill North & West, Forest Hill Road Park     0.000000   
13                           Garden District, Ryerson     0.000000   
14  Harbourfront East, Union Station, Toronto Islands     0.000000   
15                      High Park, The Junction South     0.000000   
16                     India Bazaar, The Beaches West     0.000000   
17          Kensington Market, Chinatown, Grange Park     0.000000   
18                                      Lawrence Park     0.000000   
19                           Little Portugal, Trinity     0.023810   
20                        Moore Park, Summerhill East     0.000000   
21                  North Toronto West, Lawrence Park     0.052632   
22                             Parkdale, Roncesvalles     0.000000   
23        Queen's Park, Ontario Provincial Government     0.032258   
24                          Regent Park, Harbourfront     0.021277   
25                           Richmond, Adelaide, King     0.000000   
26                                           Rosedale     0.000000   
27                                           Roselawn     0.000000   
28                                 Runnymede, Swansea     0.027778   
29                                     St. James Town     0.000000   
30                        St. James Town, Cabbagetown     0.000000   
31                                     Stn A PO Boxes     0.010204   
32                                    Studio District     0.027778   
33  Summerhill West, Rathnelly, South Hill, Forest...     0.000000   
34                The Annex, North Midtown, Yorkville     0.000000   
35                                        The Beaches     0.000000   
36                       The Danforth West, Riverdale     0.023810   
37           Toronto Dominion Centre, Design Exchange     0.000000   
38                     University of Toronto, Harbord     0.033333   

    Adult Boutique   Airport  Airport Food Court  Airport Gate  \
0         0.000000  0.000000            0.000000      0.000000   
1         0.000000  0.000000            0.000000      0.000000   
2         0.000000  0.000000            0.000000      0.000000   
3         0.000000  0.071429            0.071429      0.071429   
4         0.000000  0.000000            0.000000      0.000000   
5         0.000000  0.000000            0.000000      0.000000   
6         0.012821  0.000000            0.000000      0.000000   
7         0.000000  0.000000            0.000000      0.000000   
8         0.000000  0.000000            0.000000      0.000000   
9         0.000000  0.000000            0.000000      0.000000   
10        0.000000  0.000000            0.000000      0.000000   
11        0.000000  0.000000            0.000000      0.000000   
12        0.000000  0.000000            0.000000      0.000000   
13        0.000000  0.000000            0.000000      0.000000   
14        0.000000  0.000000            0.000000      0.000000   
15        0.000000  0.000000            0.000000      0.000000   
16        0.000000  0.000000            0.000000      0.000000   
17        0.000000  0.000000            0.000000    

In [97]:
toronto_group.shape #examnie the size of datafframe

(39, 230)

#### Let's print each neighborhood along with the top 5 most common venues

In [98]:
num_top_venues = 5

for hood in toronto_group['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_group[toronto_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.07
2  Seafood Restaurant  0.04
3         Cheese Shop  0.04
4            Beer Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3   Grocery Store  0.05
4    Climbing Gym  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Garden Center  0.06
3         Pizza Place  0.06
4                Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.21
1      Airport Lounge  0.14
2       Boat or Ferry  0.07
3             Airport  0.07
4  Airport Food Court  0.07


----Central Bay Street----
                venue  freq
0       

#### Input into data frame

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Dataframe with top 10 venues in each neighborhood

In [100]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar                Bakery              Beer Bar   
1           Coffee Shop        Breakfast Spot             Pet Store   
2           Yoga Studio            Comic Shop               Brewery   
3        Airport Lounge      Sculpture Garden   Rental Car Location   
4    Italian Restaurant                  Café        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Seafood Restaurant           Cheese Shop            Restaurant   
1                   Bar         Burrito Place            Restaurant   
2        Farmers Market  Fast Food Restaurant         Burrito Place   
3       Harbor / Marina         Boat or Ferry      Airport Terminal   
4       Bubble Tea Shop          Burger Joint           Salad Place   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0         Farmers Market         Grocery Store    Japanese Restaurant  
1           Climbing Gym         Grocery Store                 Bakery  
2             Restaurant      Recording Studio          Auto Workshop  
3                    Bar          Airport Gate     Airport Food Court  
4  Portuguese Restaurant      Ramen Restaurant      Indian Restaurant

### 14. Clustering the neigborhood

Run K-means on 5 cluster

In [101]:
kclusters = 5

toronto_grouped_clustering = toronto_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [102]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

toronto_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head()

PostalCode           Borough                                Neighbourhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster 1st Most Common Venue 2nd Most Common Venue  \
0  43.654260 -79.360636        1           Coffee Shop                  Park   
1  43.662301 -79.389494        1           Coffee Shop      Sushi Restaurant   
2  43.657162 -79.378937        1        Clothing Store           Coffee Shop   
3  43.651494 -79.375418        1                  Café           Coffee Shop   
4  43.676357 -79.293031        3                 Trail     Health Food Store   

       3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                     Bakery        Breakfast Spot            Restaurant   
1                Yoga Studio                  Bank         Smoothie Shop   
2  Middle Eastern Restaurant        Cosmetics Shop   Japanese Restaurant   
3               Cocktail Bar             Gastropub   American Restaurant   
4                        Pub              Wine Bar          Dance Studio   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                          Pub                  Café               Theater   
1                     Beer Bar        Sandwich Place         Burrito Place   
2                         Café    Italian Restaurant       Bubble Tea Shop   
3               Farmers Market                   Gym                 Hotel   
4  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
0        Cosmetics Shop             Shoe Store  
1                  Café     College Auditorium  
2  Fast Food Restaurant       Ramen Restaurant  
3      Department Store               Creperie  
4      Doner Restaurant                Dog Run

### Visualize the Cluster

In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each cluster

#### Cluster 1

In [105]:
toronto_merged.loc[toronto_merged['Cluster'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster 1st Most Common Venue 2nd Most Common Venue  \
19      Roselawn        0        Ice Cream Shop          Home Service   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
19                Garden          Dance Studio     Electronics Store   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
19  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
19      Doner Restaurant                Dog Run

#### Cluster 2

In [106]:
toronto_merged.loc[toronto_merged['Cluster'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster  \
0                           Regent Park, Harbourfront        1   
1         Queen's Park, Ontario Provincial Government        1   
2                            Garden District, Ryerson        1   
3                                      St. James Town        1   
5                                         Berczy Park        1   
6                                  Central Bay Street        1   
7                                            Christie        1   
8                            Richmond, Adelaide, King        1   
9                        Dufferin, Dovercourt Village        1   
10  Harbourfront East, Union Station, Toronto Islands        1   
11                           Little Portugal, Trinity        1   
12                       The Danforth West, Riverdale        1   
13           Toronto Dominion Centre, Design Exchange        1   
14       Brockton, Parkdale Village, Exhibition Place        1   
15                     India Bazaar, The Beaches West        1   
16                     Commerce Court, Victoria Hotel        1   
17                                    Studio District        1   
20                                   Davisville North        1   
21    Forest Hill North & West, Forest Hill Road Park        1   
22                      High Park, The Junction South        1   
23                  North Toronto West, Lawrence Park        1   
24                The Annex, North Midtown, Yorkville        1   
25                             Parkdale, Roncesvalles        1   
26                                         Davisville        1   
27                     University of Toronto, Harbord        1   
28                                 Runnymede, Swansea        1   
30          Kensington Market, Chinatown, Grange Park        1   
31  Summerhill West, Rathnelly, South Hill, Forest...        1   
32  CN Tower, King and Spadina, Railway Lands, Har...        1   
34                                     Stn A PO Boxes        1   
35                        St. James Town, Cabbagetown        1   
36             First Canadian Place, Underground city        1   
37                               Church and Wellesley        1   
38  Business reply mail Processing Centre, South C...        1   

   1st Most Common Venue          2nd Most Common Venue  \
0            Coffee Shop                           Park   
1            Coffee Shop               Sushi Restaurant   
2         Clothing Store                    Coffee Shop   
3                   Café                    Coffee Shop   
5            Coffee Shop                   Cocktail Bar   
6            Coffee Shop             Italian Restaurant   
7          Grocery Store                           Café   
8            Coffee Shop                           Café   
9                 Bakery                       Pharmacy   
10           Coffee Shop                       Aquarium   
11                   Bar                    Men's Store   
12      Greek Restaurant                    Coffee Shop   
13           Coffee Shop                          Hotel   
14                  Café                    Coffee Shop   
15                  Park           Fast Food Restaurant   
16           Coffee Shop                     Restaurant   
17           Coffee Shop                        Brewery   
20                 Hotel                           Park   
21              Bus Line                  Jewelry Store   
22                  Café             Mexican Restaurant   
23           Coffee Shop                 Clothing Store   
24        Sandwich Place                           Café   
25             Gift Shop                 Breakfast Spot   
26           Pizza Place                   Dessert Shop   
27                  Café                         Bakery   
28                  Café                    Coffee Shop   
30                  Café  Vegetarian / Vegan Restaurant   
31           Coffee Shop                            Pub   
32       Airport S

#### Cluster 3

In [107]:
toronto_merged.loc[toronto_merged['Cluster'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster 1st Most Common Venue  \
29  Moore Park, Summerhill East        2                  Park   
33                     Rosedale        2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
29            Restaurant            Playground          Tennis Court   
33            Playground                 Trail          Cupcake Shop   

          5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
29          Dumpling Restaurant            Donut Shop      Doner Restaurant   
33  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
29               Dog Run   Distribution Center         Discount Store  
33      Doner Restaurant               Dog Run    Distribution Center

#### Cluster 4

In [108]:
toronto_merged.loc[toronto_merged['Cluster'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster 1st Most Common Venue 2nd Most Common Venue  \
4   The Beaches        3                 Trail     Health Food Store   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
4                   Pub              Wine Bar          Dance Studio   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
4  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
4      Doner Restaurant                Dog Run

#### Cluster 5

In [109]:
toronto_merged.loc[toronto_merged['Cluster'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster 1st Most Common Venue 2nd Most Common Venue  \
18  Lawrence Park        4                  Park           Swim School   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
18              Bus Line    College Rec Center  Colombian Restaurant   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
18  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
18      Doner Restaurant                Dog Run